In [25]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [26]:
import importlib
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2");


In [27]:
text =("Hello, world! This is a test <|endoftext|> string to count tokens."
        "wow this is a new sentence.")

integers  = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)


[15496, 11, 995, 0, 770, 318, 257, 1332, 220, 50256, 4731, 284, 954, 16326, 13, 42773, 428, 318, 257, 649, 6827, 13]


In [28]:
strings  = tokenizer.decode(integers)
print(strings)


Hello, world! This is a test <|endoftext|> string to count tokens.wow this is a new sentence.


BPE handles, unknown vocab words, OOV(out of vocab) error by splitting the words further down, into tinier subwords or even individual characters.
now we make input-target pairs with our text.

In [33]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,text,tokenizer,maxlen,stride):
        self.input = []
        self.target = []
    
        tokenizer  = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(tokenizer), stride):
        
            inputids = tokenizer[i:i+maxlen]
            targetids = tokenizer[i+1:i+maxlen+1]
        
            self.input.append(torch.tensor(inputids))
            self.target.append(torch.tensor(targetids))

    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, idx):
        return self.input[idx], self.target[idx]




In [30]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [31]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [37]:
import torch

dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4,shuffle=False)

data_iter = iter(dataloader) #returns an object data_iter, and doing next() on it, will just give u the next next iterations in order lol.
inp,out = next(data_iter)

print(inp)
print(out)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
